# Demo GoPro Deblurring (Colab Friendly)

Notebook này cung cấp ví dụ thực nghiệm nhanh trên một (hoặc vài) cặp ảnh từ dataset GoPro để minh họa:
- Cấu trúc thư mục và cách chuẩn bị dữ liệu tối thiểu.
- Tính PSNR / SSIM cơ bản.
- Chạy một vòng tối ưu Deep Image Prior (DIP) đơn giản cho 1 ảnh.
- (Tuỳ chọn) Perceptual loss dùng VGG19.

Lưu ý quan trọng:
- Dataset GoPro đầy đủ rất lớn (khoảng hàng chục GB). Ở đây chỉ minh họa với MỘT / ÍT ảnh. Bạn nên tải subset hoặc tự chuẩn bị file mẫu.
- Đây không phải pipeline huấn luyện sản phẩm cuối – chỉ là demo thử nghiệm.
- Bạn cần GPU để tối ưu DIP nhanh hơn.

Các bước:
1. Cài đặt phụ thuộc.
2. Chuẩn bị hoặc tải ảnh mẫu (blur & sharp).
3. Viết hàm tính PSNR / SSIM, loader tiện ích.
4. Chạy baseline (giữ nguyên ảnh mờ) để có mốc.
5. Chạy DIP tối ưu mạng nhỏ sinh ảnh sắc nét.
6. (Tuỳ chọn) So sánh thêm Perceptual loss.


In [ ]:
!git clone https://github.com/ThanhQuy78/Image-Deblur-Using-Deep-Learning-.git
!mv image_deblur-using-deep-learning/* ./

In [ ]:
# 1. Cài đặt phụ thuộc (chỉ chạy trên Colab lần đầu)
# Nếu chạy local đã có môi trường thì có thể bỏ qua.
!pip -q install torch torchvision pillow matplotlib tqdm requests


In [ ]:
# 2. Helper: tải ảnh mẫu (minh họa) + hàm đánh giá
import os, io, math, requests, zipfile
from pathlib import Path
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt

DATA_DIR = Path('gopro_demo')
DATA_DIR.mkdir(exist_ok=True)
BLUR_PATH = DATA_DIR/'blur.png'
SHARP_PATH = DATA_DIR/'sharp.png'

SAMPLE_SHARP_URL = 'https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg'
# Tạo ảnh blur bằng Gaussian thay vì tải dataset thật

def download_if_needed():
    if not SHARP_PATH.exists():
        r = requests.get(SAMPLE_SHARP_URL, timeout=30)
        r.raise_for_status()
        Image.open(io.BytesIO(r.content)).convert('RGB').save(SHARP_PATH)
    if not BLUR_PATH.exists():
        sharp = Image.open(SHARP_PATH).convert('RGB')
        blur = sharp.filter(Image.Filter.GaussianBlur(radius=2.5))
        blur.save(BLUR_PATH)


def pil_to_tensor(im: Image.Image) -> torch.Tensor:
    return torch.from_numpy(np.array(im).transpose(2,0,1)).float().unsqueeze(0)/255.

def tensor_to_pil(t: torch.Tensor) -> Image.Image:
    t = t.detach().clamp(0,1)[0].cpu().numpy().transpose(1,2,0)
    return Image.fromarray((t*255).astype(np.uint8))


def compute_psnr(pred, target, data_range=1.0):
    pred = pred.float(); target = target.float()
    mse = F.mse_loss(pred, target).item()
    if mse == 0: return float('inf')
    return 20*math.log10(data_range) - 10*math.log10(mse)

def _gauss_win(ws=11, sigma=1.5, c=3):
    g = torch.arange(ws).float()-ws//2
    g = torch.exp(-(g**2)/(2*sigma**2)); g/=g.sum()
    k = (g.view(1,-1)*g.view(-1,1)).unsqueeze(0).unsqueeze(0)
    return k.repeat(c,1,1,1)

def compute_ssim(pred, target, data_range=1.0, window=11, sigma=1.5, eps=1e-8):
    pred = pred.float(); target = target.float()
    C,H,W = pred.shape[1:]
    if window>min(H,W): window=min(H,W) - (1-(min(H,W)%2))
    win = _gauss_win(window, sigma, C).to(pred.device)
    pad = window//2
    mu_x = F.conv2d(pred, win, groups=C, padding=pad)
    mu_y = F.conv2d(target, win, groups=C, padding=pad)
    mu_x2, mu_y2, mu_xy = mu_x*mu_x, mu_y*mu_y, mu_x*mu_y
    sig_x2 = F.conv2d(pred*pred, win, groups=C, padding=pad)-mu_x2
    sig_y2 = F.conv2d(target*target, win, groups=C, padding=pad)-mu_y2
    sig_xy = F.conv2d(pred*target, win, groups=C, padding=pad)-mu_xy
    C1=(0.01*data_range)**2; C2=(0.03*data_range)**2
    ssim_map=((2*mu_xy+C1)*(2*sig_xy+C2))/((mu_x2+mu_y2+C1)*(sig_x2+sig_y2+C2)+eps)
    return ssim_map.mean().item()

print('Helper loaded.')


In [ ]:
# 3. Chuẩn bị dữ liệu mẫu

download_if_needed()
sharp_img = Image.open(SHARP_PATH).convert('RGB')
blur_img  = Image.open(BLUR_PATH).convert('RGB')
sharp_t = pil_to_tensor(sharp_img)
blur_t  = pil_to_tensor(blur_img)
print('Sharp:', sharp_t.shape, 'Blur:', blur_t.shape)

fig,ax = plt.subplots(1,2, figsize=(6,3))
ax[0].imshow(sharp_img); ax[0].set_title('Sharp') ; ax[0].axis('off')
ax[1].imshow(blur_img);  ax[1].set_title('Blur')  ; ax[1].axis('off')
plt.show()


In [ ]:
# 4. Baseline: dùng trực tiếp ảnh blur để tính PSNR/SSIM so với ground truth
base_psnr = compute_psnr(blur_t, sharp_t)
base_ssim = compute_ssim(blur_t, sharp_t)
print(f'Baseline (Blur vs Sharp) -> PSNR: {base_psnr:.2f} dB | SSIM: {base_ssim:.4f}')


In [ ]:
# 5. Tối ưu DIP đơn giản (tối ưu noise để tái tạo ảnh sharp)
from models import get_net
from loss import PerceptualLoss

device = 'cuda' if torch.cuda.is_available() else 'cpu'
sharp_t = sharp_t.to(device)
input_depth = 32
net = get_net('skip', input_depth=input_depth, n_channels=3).to(device)
noise = torch.randn(1,input_depth, sharp_t.shape[-2], sharp_t.shape[-1], device=device, requires_grad=True)
optimizer = torch.optim.Adam([noise], lr=0.01)
loss_fn = PerceptualLoss(backbone_type='vgg19_features', match_mode='features', feature_dist='l1', tv_weight=1e-6)

num_iter = 400
for i in range(num_iter):
    optimizer.zero_grad()
    out = net(noise).clamp(0,1)
    loss = loss_fn(out, sharp_t)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
        with torch.no_grad():
            psnr_cur = compute_psnr(out.cpu(), sharp_t.cpu())
            print(f'Iter {i+1}/{num_iter} - Loss: {loss.item():.4f} | PSNR: {psnr_cur:.2f} dB')

restored = net(noise).clamp(0,1).detach().cpu()
psnr_final = compute_psnr(restored, sharp_t.cpu())
ssim_final = compute_ssim(restored, sharp_t.cpu())
print(f'Final -> PSNR: {psnr_final:.2f} dB | SSIM: {ssim_final:.4f}')
